In [2]:
!pip install "pillow<10.0.0"
!pip install --upgrade facenet-pytorch

  Using cached facenet_pytorch-2.6.0-py3-none-any.whl.metadata (12 kB)
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 80.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 90.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import torch
import torch.nn.functional as F
from torchvision import transforms
from facenet_pytorch import MTCNN
from PIL import Image
import json
import torch.nn as nn
# ---------- KIẾN TRÚC MÔ HÌNH ----------
class EmotionCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1), nn.BatchNorm2d(32), nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1), nn.BatchNorm2d(128), nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1), nn.BatchNorm2d(256), nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1)),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.5),
            nn.Linear(256, 7)  # FER2013 có 7 lớp cảm xúc
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

# ---------- CẤU HÌNH ----------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Danh sách cảm xúc đúng thứ tự
class_names = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']

# Load mô hình
model = EmotionCNN().to(device)
model.load_state_dict(torch.load("best_model.pt", map_location=device))
model.eval()

# MTCNN để detect khuôn mặt
mtcnn = MTCNN(keep_all=True, device=device)

# Transform giống như khi huấn luyện
transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((48, 48)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# ---------- DỰ ĐOÁN ----------
def detect_emotions(image_path):
    img = Image.open(image_path).convert("RGB")
    boxes, probs = mtcnn.detect(img)

    results = []
    if boxes is not None:
        for box in boxes:
            x1, y1, x2, y2 = map(int, box)
            width = x2 - x1
            height = y2 - y1

            # Crop khuôn mặt
            face = img.crop((x1, y1, x2, y2))
            face_tensor = transform(face).unsqueeze(0).to(device)

            # Dự đoán
            with torch.no_grad():
                output = model(face_tensor)
                prob = F.softmax(output, dim=1)
                confidence, pred = torch.max(prob, 1)

            result = {
                "bbox": [x1, y1, width, height],
                "emotion": class_names[pred.item()],
                "confidence": round(confidence.item(), 2)
            }
            results.append(result)
    return results

results = detect_emotions("/content/DSC_4903.JPG")
print(json.dumps(results, indent=2, ensure_ascii=False))

[
  {
    "bbox": [
      137,
      100,
      174,
      257
    ],
    "emotion": "happy",
    "confidence": 0.54
  }
]
